In [ ]:
import pandas as pd

In [50]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [ ]:
raw = pd.read_csv('../data/raw/ld_clump_assoc.txt',delimiter='\t')
rename = {'CpG':'cpg',
 'Top SNP':'snp',
 'Beta': 'beta',
 'SE': 'se',
 'Cis/Trans':'cistrans',
 'CpG chr': 'cpg_chr',
 'CpG pos': 'cpg_pos',
 'SNP chr': 'snp_chr',
 'SNP pos': 'snp_pos',
 'P': 'pval'}

raw = data.rename(rename, axis=1)

lst = []
for i in raw['cistrans']:
    if i == 'cis':
        lst.append(True)
    else:
        lst.append(False)
raw['cistrans'] = lst
raw.to_csv('edit_ld_clump_assoc.csv')


In [51]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5','6','7','8','9','10','11','12','13']




In [52]:
# level-based network prototype 
import pandas as pd
from pathlib import Path
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [53]:
# creating unique nodes for cpg and snp encoding level information 
# level 0 cpg --> query user
# level 1 snp --> cpg-snp
# level 1 cpg --> cpg-snp-cpg connection 

cpg_interest = ['cg13938959','cg23733394']

cpgs = []
snps = []
level = 2

for cpg in cpg_interest:
    
    start = 1
    cpgs.append({'id':cpg,
             'level':0})

    while start <= level:
        
        cpg_included = [cpg['id'] for cpg in cpgs if cpg['level'] < start]
        snps_to_include = data[data['cpg'].isin(cpg_included)]['snp'].values
        [snps.append({'id':snp,'level':start}) for snp in snps_to_include if snp not in [snp['id'] for snp in snps]]
        snps_included = [snp['id'] for snp in snps if snp['level'] == start]
        cpgs_to_include = data[data['snp'].isin(snps_included)]['cpg'].values
        [cpgs.append({'id':cpg,'level':start}) for cpg in cpgs_to_include if cpg not in [cpg['id'] for cpg in cpgs]] 
        start+= 1

edges = data[data['cpg'].isin([cpg['id'] for cpg in cpgs])]
edges = edges[edges['snp'].isin([snp['id'] for snp in snps])]

In [54]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)


In [56]:
chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5','6','7','8','9','10','11','12'] 
 

data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0) 
annot = pd.read_csv('../data/annot/cpg_annotation.csv',index_col=0)
chr_lst = chromosomeList
data['cpg_chr'] = data['cpg_chr'].apply(str)
data['snp_chr'] = data['snp_chr'].apply(str)

data = data[data['cistrans']==False]

data = data.groupby('cpg') # group data by cpgs
data = data.filter(lambda x: len(x) >= minAssoc) # filter by number of associations per cpg

data = data[data['cpg_chr'].isin(chr_lst)]
data = data[data['pval']<=1*10**-minPval]

data['cpg_pos_abs'] = data['cpg_chr'].apply(lambda cpg_chr: chromosome_distance[str(cpg_chr)]) + data['cpg_pos'] # calculate absolute distance of cpg
data['snp_pos_abs'] = data['snp_chr'].apply(lambda snp_chr: chromosome_distance[str(snp_chr)]) + data['snp_pos'] # calculate absolute distance of snp
data['dist'] = abs(data['cpg_pos_abs'] - data['snp_pos_abs']) # calculate distance between pairs
data = data[data['dist']>=minDistance] # filter by min distance

data = data.reset_index(drop=True)

data = data.merge(annot,how='left',on='cpg')
    
data['UCSC_RefGene_Name'] = data['UCSC_RefGene_Name'].fillna('no annotation')

data.drop(['MAPINFO','CHR'],axis=1,inplace=True)

response = data.to_dict('records')  



FileNotFoundError: [Errno 2] No such file or directory: '../annot/cpg_annotation.csv'

In [ ]:
data[data['UCSC_RefGene_Name'].isna()]

In [ ]:
data

In [42]:
import pandas as pd
ewas_res = pd.read_csv("../data/raw/ewascatalog-results.txt",delimiter="\t")
ewas_stud = pd.read_csv("../data/raw/ewascatalog-studies.txt",delimiter="\t")
ewas_res = ewas_res[['CpG','Chr','Pos','Gene','Beta','SE','P','StudyID']]
ewas_res['Chr'] = ewas_res['Chr']
ewas_res = ewas_res[ewas_res['CpG'].str.startswith('cg')]
ewas_stud = ewas_stud[['Trait','Exposure','N','StudyID','Age','Sex','Ancestry','Tissue']]
ewas_res.to_csv('ewas_results.csv')
ewas_stud.to_csv('ewas_stud.csv')

/var/folders/qm/j16hfspx3wqgdhdmm1rk8q_80000gp/T/ipykernel_13475/4070678898.py:2: DtypeWarning: Columns (2,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ewas_res = pd.read_csv("../data/raw/ewascatalog-results.txt",delimiter="\t")
/var/folders/qm/j16hfspx3wqgdhdmm1rk8q_80000gp/T/ipykernel_13475/4070678898.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ewas_res['Chr'] = ewas_res['Chr'].apply(str)


In [43]:
ewas_res = pd.read_csv('ewas_results.csv',index_col=0)

/var/folders/qm/j16hfspx3wqgdhdmm1rk8q_80000gp/T/ipykernel_13475/1620343873.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ewas_res = pd.read_csv('ewas_results.csv',index_col=0)


In [46]:
ewas_res['Chr'].apply(str)

0           Y
1           Y
2           Y
3           Y
4           Y
           ..
2061551    19
2061552    22
2061553    17
2061554     2
2061555    11
Name: Chr, Length: 2047909, dtype: object

In [30]:
ewas_res['CpG'].apply(str)

0          cg00035864
1          cg00050873
2          cg00455876
3          cg01707559
4          cg02012379
              ...    
2047904    cg27586378
2047905    cg27601574
2047906    cg27605300
2047907    cg27616833
2047908    cg27651452
Name: CpG, Length: 2047909, dtype: object

,CpG,Chr,Pos,Gene,Beta,SE,P,StudyID
0,cg00035864,Y,8553009.0,TTTY18,-1.081078,0.227700,2.060000e-06,27105112_hiv_infection_discovery
1,cg00050873,Y,9363356.0,TSPY4;FAM197Y2,-1.746455,0.346400,4.600000e-07,27105112_hiv_infection_discovery
2,cg00455876,Y,9385539.0,-,-1.953244,0.289100,1.410000e-11,27105112_hiv_infection_discovery
3,cg01707559,Y,6778695.0,TBL1Y,1.151949,0.272100,2.300000e-05,27105112_hiv_infection_discovery
4,cg02012379,Y,22917894.0,RPS4Y2,-1.751271,0.242100,4.720000e-13,27105112_hiv_infection_discovery
...,...,...,...,...,...,...,...,...
2061551,cg27586378,19,49063066.0,SULT2B1,-0.000560,0.000130,2.800000e-05,Fernandez-Jimenez-N_maternal_pre-pregnancy_bod...
2061552,cg27601574,22,31498863.0,SMTN,-0.000830,0.000180,4.900000e-06,Fernandez-Jimenez-N_maternal_pre-pregnancy_bod...
2061553,cg27605300,17,79360456.0,-,-0.000170,0.000036,8.300000e-07,Fernandez-Jimenez-N_maternal_pre-pregnancy_bod...
2061554,cg27616833,2,106060221.0,-,-0.000660,0.000140,2.300000e-06,Fernandez-Jimenez-N_maternal_pre-pregnancy_bod...
